# **1. 데이터 및 라이브러리 불러오기**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
from tqdm import tqdm

from konlpy.tag import Okt
import pickle as plk

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv('./submission/sample_submission.csv')

# **2. 데이터 EDA**

In [3]:
train.head()

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
0,0,2016,농업기초기반연구,농촌진흥청,신규,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24
1,1,2019,이공학학술연구기반구축(R&D),교육부,신규,지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0
2,2,2016,중소기업기술혁신개발,중소기업청,신규,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0
3,3,2018,창업성장기술개발(R&D),중소벤처기업부,신규,창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0
4,4,2016,이공학개인기초연구지원,교육부,계속,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0


In [4]:
train.isnull().sum()

index           0
제출년도            0
사업명             0
사업_부처명          0
계속과제여부          0
내역사업명           0
과제명             0
요약문_연구목표     3002
요약문_연구내용     3001
요약문_기대효과     3051
요약문_한글키워드    3028
요약문_영문키워드    3087
label           0
dtype: int64

In [5]:
train = train.fillna('NAN')
train.isnull().sum()

index        0
제출년도         0
사업명          0
사업_부처명       0
계속과제여부       0
내역사업명        0
과제명          0
요약문_연구목표     0
요약문_연구내용     0
요약문_기대효과     0
요약문_한글키워드    0
요약문_영문키워드    0
label        0
dtype: int64

In [6]:
test.head()

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드
0,174304,2016,경제협력권산업육성,산업통상자원부,신규,자동차융합부품,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...,○ 차체 점용접부의 품질 검사를 위한 64채널 무선 기반 C-Scan 탐촉자 개발\...,○ 1차년도\n\n . 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시...,○ 기술적 파급효과\n\n - 본 연구에서 개발된 R-FSSW 접합 기술은 기존 ...,"마찰교반점용접, 비파괴 검사, 초음파 탐상, 씨 스캔, 용접 품질 평가","Friction Stir Spot Welding, Non-destructive ev..."
1,174305,2018,개인기초연구(과기정통부)(R&D),과학기술정보통신부,계속,신진연구(총연구비5천이상~1.5억이하),다입자계를 묘사하는 편미분방정식에 대한 연구,자연계에는 입자의 개수가 아주 큰 다양한 다입자계가 존재한다. 이런 다입자계의 효...,연구과제1. 무한입자계의 동역학 / 작용소(operator) 방정식에 대한 연구\n...,본 연구는 물리학에서 중요한 대상인 다입자계를 묘사하는 모델방정식의 정당성을 보장하...,"다체계 방정식,동역학의 안정성,양자역학,고전역학,평균장 극한,고전극한,비상대론적 극한","many particle system,stability of dynamics,qua..."
2,174306,2017,개인기초연구(미래부),과학기술정보통신부,계속,자유공모,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발,"저출생체중아는 정상 체중 출생아에 비하여 운동, 정신, 행동 및 정서발달장애가 빈번...",본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...,본 연구는 저출생체중아 가족 대상의 지역사회기반의 추후관리 사업에 대한 구체적인 모...,"저출생체중아,추후관리,조기개입,자조모임,가정방문","lowbirthweightinfant,followup,earlyinterventio..."
3,174307,2018,산학연협력기술개발(R&D),중소벤처기업부,신규,전략 협력(지역유망),폴리싱용 5축 CNC 정밀 마이크로 시스템 개발,최근 초정밀 공작기계 시장의 트렌드를 살펴보면 다음과 같다.\n\n\n(1) 자동차...,(1) 0.5~1㎛의 가공정밀도(면조도) 구현\n\n\n - 국내에서는 연삭기를 제...,(1) 가공 공정 및 소요시간 단축\n\n\n 초정밀금형 가공시 정삭 후 추가적...,"초정밀가공,폴리싱 가공,마이크로,경면가공,난삭재가공","superprecision,hiprecision,micro,mirror like f..."
4,174308,2019,이공학학술연구기반구축(R&D),교육부,계속,지역대학우수과학자지원사업(1년~5년),다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델 개발,본 연구의 최종목표는 다결정 금속재료의 미세조직구조를 고려한 확률론적 응력부식균열(...,"총 연구기간은 3년으로, 연차별 연구내용 및 범위는 다음과 같다.\n\n ∙ 1차...",본 연구의 중요성 및 활용가능성은 다음과 같다.\n\n ∙ 기존의 재료전체 ave...,"결정구조,응력부식균열,확률론적모델","Crystalline structure,Stress corrosion crackin..."


In [7]:
test.isnull().sum()

index          0
제출년도           0
사업명            0
사업_부처명         0
계속과제여부         0
내역사업명          0
과제명            0
요약문_연구목표     755
요약문_연구내용     755
요약문_기대효과     761
요약문_한글키워드    760
요약문_영문키워드    772
dtype: int64

In [8]:
test = test.fillna('NAN')
test.isnull().sum()

index        0
제출년도         0
사업명          0
사업_부처명       0
계속과제여부       0
내역사업명        0
과제명          0
요약문_연구목표     0
요약문_연구내용     0
요약문_기대효과     0
요약문_한글키워드    0
요약문_영문키워드    0
dtype: int64

In [9]:
sample_submission.head(6)

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
5,174309,0


In [10]:
#데이터 구조 파악
print(train.shape)
print(test.shape)
print(sample_submission.shape)

(174304, 13)
(43576, 12)
(43576, 2)


In [11]:
#심각한 불균형 데이터임을 알 수 있습니다.
train.label.value_counts(sort=False)/len(train)

0     0.817945
1     0.007234
2     0.001578
3     0.000820
4     0.000327
5     0.009742
6     0.000447
7     0.000648
8     0.001945
9     0.000608
10    0.003775
11    0.001147
12    0.001538
13    0.003299
14    0.009592
15    0.000947
16    0.002903
17    0.000884
18    0.008893
19    0.028330
20    0.006076
21    0.002846
22    0.000849
23    0.010556
24    0.020195
25    0.004647
26    0.001813
27    0.003557
28    0.002576
29    0.005898
30    0.001342
31    0.005290
32    0.001492
33    0.003058
34    0.003001
35    0.001669
36    0.006081
37    0.001526
38    0.001503
39    0.001159
40    0.002530
41    0.000384
42    0.000293
43    0.002014
44    0.000522
45    0.006523
Name: label, dtype: float64

In [12]:
len(train['제출년도'].value_counts())

4

In [13]:
len(train['사업_부처명'].value_counts())

28

In [14]:
len(train['계속과제여부'].value_counts())

2

In [15]:
train_dum = pd.get_dummies(train, columns = ['제출년도', '사업_부처명', '계속과제여부'])
train_dum.head()

,index,사업명,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label,...,사업_부처명_중소기업청,사업_부처명_중소벤처기업부,사업_부처명_특허청,사업_부처명_해양경찰청,사업_부처명_해양수산부,사업_부처명_행정안전부,사업_부처명_행정자치부,사업_부처명_환경부,계속과제여부_계속,계속과제여부_신규
0,0,농업기초기반연구,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,(가) 외래 및 돌발해충의 발생조사 및 종 동정\n\n\n ○ 대상해충 : 최...,"○ 새로운 돌발 및 외래해충의 신속, 정확한 동정법 향상\n\n\n○ 돌발 및 외래...","뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",24,...,0,0,0,0,0,0,0,0,0,1
1,1,이공학학술연구기반구축(R&D),지역대학우수과학자지원사업(1년~5년),대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,1차년도\n1) Microarray를 통한 선천적 TRAIL 내성 표적 후보 유전자...,"1) TRAIL 내성 특이적 표적분자를 발굴하고, 이를 이용한 TRAIL 효과 증진...","대장암,항암제 내성,세포사멸,유전자발굴","TRAIL,Colorectal cancer,TRAIL resistance,Apopt...",0,...,0,0,0,0,0,0,0,0,0,1
2,2,중소기업기술혁신개발,혁신기업기술개발,비목질계 셀룰로오스 식물자원을 활용한 기능성 부직포 및 고부가가치 뷰티케어 faci...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n* 소비자 및 바...,* 식물계자원 정련 및 최적 신서란 파이버 기초연구 개발\n\n\n- Tencel/...,* 국내 독자적인 비목질계 셀룰로오스 자원의 파이버 및 부직포 제조 등의 기술 확보...,"기능성 샐룰로오스 파이버,천연섬유,기능성 부직포,뷰티&amp;케어 제품,미용 솜","functional cellulose fiber,natural fiber,funct...",0,...,1,0,0,0,0,0,0,0,0,1
3,3,창업성장기술개발(R&D),창업사업화연계과제,소화기 암 진단용 분자영상 형광프로브 개발,# 암특이적 바이오마커 발굴 및 바이오마커에 대한 프로브 개발\n\n\n1) 소화...,# 소화기 암 진단용 분자영상 형광프로브 개발\n\n\n- 국소 도포형 소화기 암 ...,# 암 진단기술의 차별성: 소화기 암 특이 프로브 개발\n\n\n- 최근 체외진단시...,"분자 진단,형광 조영제,프로브,항체,대장암","Molecular diagnosis,Fluorescence,probe,antibod...",0,...,0,1,0,0,0,0,0,0,0,1
4,4,이공학개인기초연구지원,기본연구지원,위암환자의 항암제반응예측을 위한 BRCA 발현검사,"수술이 불가능한 위암환자는 생존기간은 10개월 안팎에 지나지 않고, 항암화학요법에 ...",-In situ hybridization 검사의 정확성을 확인하기 위해 위암세포주 ...,-본 연구는 파라핀보관조직에서 in situ hybridization로 mRNA 및...,"BRCA,제자리부합법,조직미세배열,마이크로RNA,위암,항암제반응,젊은 연령/가족성 위암","BRCA,Insituhybridization,tissuemicroarray,micr...",0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
test_dum = pd.get_dummies(test, columns = ['제출년도', '사업_부처명', '계속과제여부'])
test_dum.head()

,index,사업명,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,제출년도_2016,...,사업_부처명_중소기업청,사업_부처명_중소벤처기업부,사업_부처명_특허청,사업_부처명_해양경찰청,사업_부처명_해양수산부,사업_부처명_행정안전부,사업_부처명_행정자치부,사업_부처명_환경부,계속과제여부_계속,계속과제여부_신규
0,174304,경제협력권산업육성,자동차융합부품,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...,○ 차체 점용접부의 품질 검사를 위한 64채널 무선 기반 C-Scan 탐촉자 개발\...,○ 1차년도\n\n . 개발 탐촉 시스템의 성능 평가 위한 표준 시편 제작 시...,○ 기술적 파급효과\n\n - 본 연구에서 개발된 R-FSSW 접합 기술은 기존 ...,"마찰교반점용접, 비파괴 검사, 초음파 탐상, 씨 스캔, 용접 품질 평가","Friction Stir Spot Welding, Non-destructive ev...",1,...,0,0,0,0,0,0,0,0,0,1
1,174305,개인기초연구(과기정통부)(R&D),신진연구(총연구비5천이상~1.5억이하),다입자계를 묘사하는 편미분방정식에 대한 연구,자연계에는 입자의 개수가 아주 큰 다양한 다입자계가 존재한다. 이런 다입자계의 효...,연구과제1. 무한입자계의 동역학 / 작용소(operator) 방정식에 대한 연구\n...,본 연구는 물리학에서 중요한 대상인 다입자계를 묘사하는 모델방정식의 정당성을 보장하...,"다체계 방정식,동역학의 안정성,양자역학,고전역학,평균장 극한,고전극한,비상대론적 극한","many particle system,stability of dynamics,qua...",0,...,0,0,0,0,0,0,0,0,1,0
2,174306,개인기초연구(미래부),자유공모,저출생체중아 가족을 위한 지역사회중심의 퇴원후 추후관리프로그램 개발,"저출생체중아는 정상 체중 출생아에 비하여 운동, 정신, 행동 및 정서발달장애가 빈번...",본 연구는 퇴원 후 저출생체중아의 퇴원 후 추후관리 프로그램 중 가정방문 모델과 가...,본 연구는 저출생체중아 가족 대상의 지역사회기반의 추후관리 사업에 대한 구체적인 모...,"저출생체중아,추후관리,조기개입,자조모임,가정방문","lowbirthweightinfant,followup,earlyinterventio...",0,...,0,0,0,0,0,0,0,0,1,0
3,174307,산학연협력기술개발(R&D),전략 협력(지역유망),폴리싱용 5축 CNC 정밀 마이크로 시스템 개발,최근 초정밀 공작기계 시장의 트렌드를 살펴보면 다음과 같다.\n\n\n(1) 자동차...,(1) 0.5~1㎛의 가공정밀도(면조도) 구현\n\n\n - 국내에서는 연삭기를 제...,(1) 가공 공정 및 소요시간 단축\n\n\n 초정밀금형 가공시 정삭 후 추가적...,"초정밀가공,폴리싱 가공,마이크로,경면가공,난삭재가공","superprecision,hiprecision,micro,mirror like f...",0,...,0,1,0,0,0,0,0,0,0,1
4,174308,이공학학술연구기반구축(R&D),지역대학우수과학자지원사업(1년~5년),다결정재료의 미세조직구조를 고려한 확률론적 응력부식균열 생성예측모델 개발,본 연구의 최종목표는 다결정 금속재료의 미세조직구조를 고려한 확률론적 응력부식균열(...,"총 연구기간은 3년으로, 연차별 연구내용 및 범위는 다음과 같다.\n\n ∙ 1차...",본 연구의 중요성 및 활용가능성은 다음과 같다.\n\n ∙ 기존의 재료전체 ave...,"결정구조,응력부식균열,확률론적모델","Crystalline structure,Stress corrosion crackin...",0,...,0,0,0,0,0,0,0,0,1,0


# **3. 데이터 전처리**

In [17]:
train_dum.columns

Index(['index', '사업명', '내역사업명', '과제명', '요약문_연구목표', '요약문_연구내용', '요약문_기대효과',
       '요약문_한글키워드', '요약문_영문키워드', 'label', '제출년도_2016', '제출년도_2017',
       '제출년도_2018', '제출년도_2019', '사업_부처명_경찰청', '사업_부처명_과학기술정보통신부',
       '사업_부처명_교육부', '사업_부처명_국민안전처', '사업_부처명_국토교통부', '사업_부처명_기상청',
       '사업_부처명_기획재정부', '사업_부처명_농림축산식품부', '사업_부처명_농촌진흥청', '사업_부처명_다부처',
       '사업_부처명_문화재청', '사업_부처명_문화체육관광부', '사업_부처명_미래창조과학부', '사업_부처명_범부처 사업',
       '사업_부처명_보건복지부', '사업_부처명_산림청', '사업_부처명_산업통상자원부', '사업_부처명_소방청',
       '사업_부처명_식품의약품안전처', '사업_부처명_원자력안전위원회', '사업_부처명_중소기업청', '사업_부처명_중소벤처기업부',
       '사업_부처명_특허청', '사업_부처명_해양경찰청', '사업_부처명_해양수산부', '사업_부처명_행정안전부',
       '사업_부처명_행정자치부', '사업_부처명_환경부', '계속과제여부_계속', '계속과제여부_신규'],
      dtype='object')

In [25]:
#해당 baseline 에서는 과제명 columns만 활용했습니다.
#다채로운 변수 활용법으로 성능을 높여주세요!
train_dum = train_dum[['제출년도_2016', '제출년도_2017', '제출년도_2018', '제출년도_2019',
                       '사업_부처명_경찰청', '사업_부처명_과학기술정보통신부', '사업_부처명_교육부', '사업_부처명_국민안전처',
                       '사업_부처명_국토교통부', '사업_부처명_기상청', '사업_부처명_기획재정부', '사업_부처명_농림축산식품부',
                       '사업_부처명_농촌진흥청', '사업_부처명_다부처', '사업_부처명_문화재청', '사업_부처명_문화체육관광부',
                       '사업_부처명_미래창조과학부', '사업_부처명_범부처 사업', '사업_부처명_보건복지부', '사업_부처명_산림청',
                       '사업_부처명_산업통상자원부', '사업_부처명_소방청', '사업_부처명_식품의약품안전처',
                       '사업_부처명_원자력안전위원회', '사업_부처명_중소기업청', '사업_부처명_중소벤처기업부', '사업_부처명_특허청',
                       '사업_부처명_해양경찰청', '사업_부처명_해양수산부', '사업_부처명_행정안전부', '사업_부처명_행정자치부',
                       '사업_부처명_환경부', '계속과제여부_계속', '계속과제여부_신규']]

test_dum = test_dum[['제출년도_2016', '제출년도_2017', '제출년도_2018', '제출년도_2019',
                     '사업_부처명_경찰청', '사업_부처명_과학기술정보통신부', '사업_부처명_교육부', '사업_부처명_국민안전처',
                     '사업_부처명_국토교통부', '사업_부처명_기상청', '사업_부처명_기획재정부', '사업_부처명_농림축산식품부',
                     '사업_부처명_농촌진흥청', '사업_부처명_다부처', '사업_부처명_문화재청', '사업_부처명_문화체육관광부',
                     '사업_부처명_미래창조과학부', '사업_부처명_범부처 사업', '사업_부처명_보건복지부', '사업_부처명_산림청',
                     '사업_부처명_산업통상자원부', '사업_부처명_소방청', '사업_부처명_식품의약품안전처',
                     '사업_부처명_원자력안전위원회', '사업_부처명_중소기업청', '사업_부처명_중소벤처기업부', '사업_부처명_특허청',
                     '사업_부처명_해양경찰청', '사업_부처명_해양수산부', '사업_부처명_행정안전부', '사업_부처명_행정자치부',
                     '사업_부처명_환경부', '계속과제여부_계속', '계속과제여부_신규']]

In [26]:
train_dum.head()

,제출년도_2016,제출년도_2017,제출년도_2018,제출년도_2019,사업_부처명_경찰청,사업_부처명_과학기술정보통신부,사업_부처명_교육부,사업_부처명_국민안전처,사업_부처명_국토교통부,사업_부처명_기상청,...,사업_부처명_중소기업청,사업_부처명_중소벤처기업부,사업_부처명_특허청,사업_부처명_해양경찰청,사업_부처명_해양수산부,사업_부처명_행정안전부,사업_부처명_행정자치부,사업_부처명_환경부,계속과제여부_계속,계속과제여부_신규
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
test_dum.head()

,제출년도_2016,제출년도_2017,제출년도_2018,제출년도_2019,사업_부처명_경찰청,사업_부처명_과학기술정보통신부,사업_부처명_교육부,사업_부처명_국민안전처,사업_부처명_국토교통부,사업_부처명_기상청,...,사업_부처명_중소기업청,사업_부처명_중소벤처기업부,사업_부처명_특허청,사업_부처명_해양경찰청,사업_부처명_해양수산부,사업_부처명_행정안전부,사업_부처명_행정자치부,사업_부처명_환경부,계속과제여부_계속,계속과제여부_신규
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
def text_preprocessing(text_list):
    
    stopwords = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
    tokenizer = Okt()
    token_list = []
    
    for text in tqdm(text_list):
        text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
        #text = re.sub('[^A-Za-z0-9가-힣]', '', text)
        #text = text.lower()
        token = tokenizer.morphs(text, stem=True) 
        token = [t for t in token if t not in stopwords or type(t) != float]
        token_list.append(token)
        
    return token_list

In [22]:
train_tokenized_title = text_preprocessing(train['과제명'])
train_tokenized_keyword = text_preprocessing(train['요약문_한글키워드'])

test_tokenized_title = text_preprocessing(test['과제명'])
test_tokenized_keyword = text_preprocessing(test['요약문_한글키워드'])

100%|████████████████████████████████████████████████████████████████████████████| 43576/43576 [22:06<00:00, 32.85it/s]


In [23]:
len(train_tokenized_title)

174304

In [24]:
len(test_tokenized_title)

43576

import pickle as pkl

with open('./data/train_tokenized_title.pkl','wb') as f:
    pkl.dump(train_tokenized_title, f, protocol=pkl.HIGHEST_PROTOCOL)

with open('./data/train_tokenized_keyword.pkl','wb') as f:
    pkl.dump(train_tokenized_keyword, f, protocol=pkl.HIGHEST_PROTOCOL)

with open('./data/test_tokenized_title.pkl','wb') as f:
    pkl.dump(test_tokenized_title, f, protocol=pkl.HIGHEST_PROTOCOL)

with open('./data/test_tokenized_keyword.pkl','wb') as f:
    pkl.dump(test_tokenized_keyword, f, protocol=pkl.HIGHEST_PROTOCOL)

"""with open('./tokenizer/train_tokenized_title.pkl','rb') as f:
    train_tokenized_title = pkl.load(f)
    
f.close()"""

In [56]:
total_token = train_tokenized_title + train_tokenized_keyword
len(total_token)

348608

In [40]:
import itertools
total_token = list(itertools.chain(*total_token))

In [63]:
countvectorizer = CountVectorizer(tokenizer = lambda x: x, lowercase=False, max_features=10000)
countvectorizer.fit(total_token)

C:\Users\user\.conda\envs\green\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(lowercase=False, max_features=10000,
                tokenizer=<function <lambda> at 0x0000021A9C0F8CA8>)

In [64]:
countvectorizer.vocabulary_

{'유전': 6051,
 '정보': 7109,
 '를': 2188,
 '활용': 9860,
 '한새': 9396,
 '로운': 2136,
 '해충': 9561,
 '동정': 1869,
 '기술': 1095,
 '개발': 257,
 '대장암': 1670,
 '의': 6173,
 '내': 1258,
 '성': 4107,
 '표적': 9112,
 '인자': 6430,
 '발굴': 2909,
 '및': 2768,
 '반응': 2882,
 '예측': 5577,
 '유전자': 6056,
 '지도': 7565,
 '구축': 868,
 '에': 5311,
 '관': 698,
 '한': 9368,
 '연구': 5442,
 '지다': 7563,
 '셀룰로오스': 4261,
 '식': 4767,
 '물자원': 2678,
 '을': 6130,
 '기능': 1051,
 '부직포': 3364,
 '고': 488,
 '부가가치': 3320,
 '뷰티': 3485,
 '케어': 8304,
 '제품': 7254,
 '소화기': 4341,
 '암': 5115,
 '진단': 7682,
 '용': 5736,
 '분자영상': 3433,
 '형광': 9687,
 '프로': 9174,
 '브': 3486,
 '위암': 5938,
 '환자': 9848,
 '항암제': 9500,
 '위': 5924,
 '발현': 2946,
 '검사': 328,
 '국제': 887,
 '핵융합': 9573,
 '재료': 6783,
 '조사': 7292,
 '시설': 4730,
 '안정': 5080,
 '적': 6844,
 '운전': 5851,
 '강도': 211,
 '빔': 3623,
 '물리': 2665,
 '마이크로': 2313,
 '시스': 4732,
 '적용': 6860,
 '옥': 5626,
 '내외': 1275,
 '겸용': 400,
 '광케이블': 774,
 '임': 6499,
 '상': 3815,
 '오믹스': 5593,
 '통합': 8658,
 '개방': 260,
 '형': 9685,
 '플랫폼': 923

In [65]:
len(countvectorizer.vocabulary_)

10000

In [67]:
train_title_countvec = countvectorizer.transform(train_tokenized_title).toarray()
train_keyword_countvec = countvectorizer.transform(train_tokenized_keyword).toarray()

test_title_countvec = countvectorizer.transform(test_tokenized_title).toarray()
test_keyword_countvec = countvectorizer.transform(test_tokenized_keyword).toarray()

MemoryError: Unable to allocate 13.0 GiB for an array with shape (174304, 10000) and data type int64

In [ ]:
train_title_countvec_df = pd.DataFrame(train_title_countvec)
train_keyword_countvec_df = pd.DataFrame(train_keyword_countvec)
test_title_countvec_df = pd.DataFrame(test_title_countvec)
test_keyword_countvec_df = pd.DataFrame(test_keyword_countvec)

train_title_countvec_df = train_title_countvec_df.add_prefix('title_')
train_keyword_countvec_df = train_keyword_countvec_df.add_prefix('key_')
test_title_countvec_df = test_title_countvec_df.add_prefix('title_')
test_keyword_countvec_df = test_keyword_countvec_df.add_prefix('key_')

In [ ]:
train_keyword_countvec_df

In [ ]:
train_title_countvec_df.head()

In [ ]:
train_keyword_countvec_df.head()

In [ ]:
test_title_countvec_df.head()

In [ ]:
test_keyword_countvec_df.head()

In [ ]:
X_train = pd.concat([train_dum, train_title_countvec_df, train_keyword_countvec_df], axis=1)
X_train

In [ ]:
X_test = pd.concat([test_dum, test_title_countvec_df, test_keyword_countvec_df], axis=1)
X_test

# **4. 모델링**

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(X_train, train['label'], test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=1000, random_state=42)

forest.fit(train_x, train_y)

In [ ]:
pred = forest.predict(val_x)
pred

In [ ]:
accuracy = accuracy_score(val_y, pred)
accuracy

In [ ]:
f1 = f1_score(val_y, pred, average='macro')
f1

# **5. 예측 및 제출**

In [ ]:
test_pred = forest.predict(X_test)

In [ ]:
sample_submission['label'] = test_pred

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv('./submission/baseline_rf_v2.csv', index=False)

In [ ]:
sample_submission.describe()